## Global Setup and Library Imports

In [ ]:
# ------------------------------------------
# Set global random seed for reproducibility
# ------------------------------------------
import os
os.environ['PYTHONHASHSEED'] = '1'

import random as rn
import numpy as np
from numpy.random import seed
seed(1)
rn.seed(1)

# ------------------------------------------
# Suppress warnings for cleaner outputs
# ------------------------------------------
import warnings
warnings.filterwarnings("ignore")

# ------------------------------------------
# General-purpose libraries
# ------------------------------------------
import sys
import glob
import gc
import ast
import itertools

# ------------------------------------------
# Data handling and visualization
# ------------------------------------------
import pandas as pd
import openpyxl  # Ensure openpyxl is installed and imported

# ------------------------------------------
# Feature extraction and preprocessing
# ------------------------------------------
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import kurtosis, skew
from numpy import sqrt, argmax

# ------------------------------------------
# Dimensionality reduction & visualization
# ------------------------------------------
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# ------------------------------------------
# Model selection and evaluation
# ------------------------------------------
from sklearn.model_selection import (
    StratifiedShuffleSplit,
    StratifiedKFold,
    KFold,
    LeaveOneGroupOut,
    cross_val_predict,
    cross_val_score,
    validation_curve,
    GridSearchCV,
    learning_curve
)

# ------------------------------------------
# Classifiers
# ------------------------------------------
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import svm, tree
from sklearn.svm import SVC, LinearSVC

# ------------------------------------------
# Evaluation metrics
# ------------------------------------------
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix,
    precision_recall_fscore_support
)


# ------------------------------------------
# Feature selection
# ------------------------------------------
from sklearn.feature_selection import (
    SelectKBest,
    chi2,
    SelectPercentile,
    f_classif,
    RFECV
)

# ------------------------------------------
# Sklearn utilities
# ------------------------------------------
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from collections import Counter
from sklearn.preprocessing import LabelBinarizer



# ------------------------------------------
# Helper function to extract subject ID
# ------------------------------------------
def obtain_subject(f):
    """
    Extracts the subject ID (e.g., 'M1', 'F4') from the filename.
    
    Assumes filenames follow a pattern like:
    'Other_M1_Glass_Sit_12_02-11-2020_11-04-49.txt'

    Parameters:
        f (str): Full path or filename of the data file.

    Returns:
        str: The subject ID string.
    """
    filename = os.path.basename(f)
    parts = filename.split('_')
    return parts[1]  # e.g., "M1", "F2", etc.



## Data Variables

    
initial component list, number_initial_components:
Defines the complete list of available signals and selects how many of them to consider for processing.

best_features_number:
Sets the maximum number of features to process (e.g., 486 features for 9 signals).

Dnumber_of_segments:
Determines the number of subwindows (segments) each data example is divided into.

n_times:
Sets how many times the experiment is repeated, with results being averaged.


In [2]:
# ------------------------------------------------------------
# Signal Configuration
# ------------------------------------------------------------

# Number of signal components to use (e.g., 3 for XYZ, 9 for all)
number_initial_components = 9

# Full list of available raw signal components
initial_componet_list = ["x", "y", "z", "gyrox", "gyroy", "gyroz", "pitch", "roll", "yaw"]

# Subset of components actually used for feature extraction
initial_data_reduced_names = initial_componet_list[:number_initial_components]

# Display signal configuration
print("All signal components:", initial_componet_list)
print("Selected components:", initial_data_reduced_names)

# ------------------------------------------------------------
# Feature Selection Configuration
# ------------------------------------------------------------

# Maximum number of features to retain (e.g., 486 for 9 components)
# - Max 162 for XYZ (3 signals)
# - Max 54 for X only
best_features_number = 486

# ------------------------------------------------------------
# Segmentation Parameters
# ------------------------------------------------------------
number_of_segments = 5

# ------------------------------------------------------------
# Dataset Path Configuration
# ------------------------------------------------------------

# Relative path to dataset folder
root = './OHM_Dataset_3class/'  # Regular Sampling

All signal components: ['x', 'y', 'z', 'gyrox', 'gyroy', 'gyroz', 'pitch', 'roll', 'yaw']
Selected components: ['x', 'y', 'z', 'gyrox', 'gyroy', 'gyroz', 'pitch', 'roll', 'yaw']


## Activity Mapping and Feature Extraction

Defines the activity label mapping (assigning numeric codes to each activity), loads and processes data files for each activity class using the gather() function, and implements feature extraction routines. It includes helper functions for median filtering with a sliding window (strided_app()) and for extracting statistical features (both over the whole signal and segmented subwindows) from each file, while also appending class and subject identifiers.

In [3]:
# ------------------------------------------------------------
# Activity Class Mapping
# ------------------------------------------------------------
# Dictionary mapping activity names to numeric labels
wrist_class = {
    'Other': 0,
    'Drink_glass': 1,
    'Drink_bottle': 2
}

# Sorted list of class labels
wrist_labels = sorted(wrist_class, key=lambda x: x[1], reverse=True)
labels = wrist_labels

print("Activity labels:", wrist_labels)

# ------------------------------------------------------------
# Load and Process All Files for Each Class
# ------------------------------------------------------------
def gather(class_dict, split_index, segments):
    """
    Imports .txt files per class, computes features, and returns a combined DataFrame.
    """
    df = []
    for c in class_dict.keys():
        f = glob.glob(root + c + '/*')  # Get all files in class folder
        d = pd.DataFrame(reformat(f, cls=c, split_index=split_index, segments=segments))  # Extract features
        print("Processed class:", c)
        df.append(d)
    return pd.concat(df)

# ------------------------------------------------------------
# Median Filtering with Striding (used for noise smoothing)
# ------------------------------------------------------------
def strided_app(a, L, S):
    """
    Applies a sliding window (stride trick) for median filtering.
    L: window length, S: stride step.
    """
    nrows = ((a.size - L) // S) + 1
    n = a.strides[0]
    return np.lib.stride_tricks.as_strided(a, shape=(nrows, L), strides=(S * n, n))

# ------------------------------------------------------------
# Extract Features from Each File
# ------------------------------------------------------------
def reformat(files, cls, split_index, segments):
    """
    Extracts statistical features from sensor data, optionally segmented.
    """
    big_list = []
    
    for f in files:
        subject_id = obtain_subject(f)
        
        # Read signal data with predefined columns
        data = pd.read_csv(f, sep=',', header=None, names=[
            'x', 'y', 'z', 'gyrox', 'gyroy', 'gyroz', 'pitch', 'roll', 'yaw'
        ])

        # Extract individual axis columns
        df_x, df_y, df_z = data.iloc[:, 0:1], data.iloc[:, 1:2], data.iloc[:, 2:3]
        df_gyrox, df_gyroy, df_gyroz = data.iloc[:, 3:4], data.iloc[:, 4:5], data.iloc[:, 5:6]
        df_pitch, df_roll, df_yaw = data.iloc[:, 6:7], data.iloc[:, 7:8], data.iloc[:, 8:9]

        # Apply median filtering (window = 3, stride = 1)
        x = np.median(strided_app(df_x.values.flatten(), 3, 1), axis=1)
        y = np.median(strided_app(df_y.values.flatten(), 3, 1), axis=1)
        z = np.median(strided_app(df_z.values.flatten(), 3, 1), axis=1)
        gyrox = np.median(strided_app(df_gyrox.values.flatten(), 3, 1), axis=1)
        gyroy = np.median(strided_app(df_gyroy.values.flatten(), 3, 1), axis=1)
        gyroz = np.median(strided_app(df_gyroz.values.flatten(), 3, 1), axis=1)
        pitch = np.median(strided_app(df_pitch.values.flatten(), 3, 1), axis=1)
        roll = np.median(strided_app(df_roll.values.flatten(), 3, 1), axis=1)
        yaw = np.median(strided_app(df_yaw.values.flatten(), 3, 1), axis=1)

        # Concatenate all signals into one DataFrame
        data_all = pd.concat([
            df_x.reset_index(drop=True), df_y.reset_index(drop=True), df_z.reset_index(drop=True),
            df_gyrox.reset_index(drop=True), df_gyroy.reset_index(drop=True), df_gyroz.reset_index(drop=True),
            df_pitch.reset_index(drop=True), df_roll.reset_index(drop=True), df_yaw.reset_index(drop=True)
        ], axis=1)

        # Select only the first N components
        data_reduced = data_all.iloc[:, :number_initial_components]

        # Split the time series into segments
        data_split = np.array_split(data_reduced, split_index)

        appended_features = []

        # Whole-window features
        features_whole = pd.concat([
            data_reduced.mean(axis=0).rename(index=lambda x: 'mean_' + x),
            data_reduced.std(axis=0).rename(index=lambda x: 'std_' + x),
            data_reduced.median(axis=0).rename(index=lambda x: 'median_' + x),
            data_reduced.sub(data_reduced.mean()).abs().mean().rename(index=lambda x: 'mad_' + x),
            data_reduced.max(axis=0).rename(index=lambda x: 'max_' + x),
            data_reduced.kurtosis(axis=0).rename(index=lambda x: 'kur_' + x),
            data_reduced.skew(axis=0).rename(index=lambda x: 'skw_' + x),
            data_reduced.var(axis=0).rename(index=lambda x: 'var_' + x),
            data_reduced.min(axis=0).rename(index=lambda x: 'min_' + x)
        ])

        # Segment-wise features
        for i in range(split_index):
            features = pd.concat([
                data_split[i].mean(axis=0).rename(index=lambda x: f'mean_{x}_{i}'),
                data_split[i].std(axis=0).rename(index=lambda x: f'std_{x}_{i}'),
                data_split[i].median(axis=0).rename(index=lambda x: f'median_{x}_{i}'),
                data_split[i].sub(data_split[i].mean()).abs().mean().rename(index=lambda x: f'mad_{x}_{i}'),
                data_split[i].max(axis=0).rename(index=lambda x: f'max_{x}_{i}'),
                data_split[i].kurtosis(axis=0).rename(index=lambda x: f'kur_{x}_{i}'),
                data_split[i].skew(axis=0).rename(index=lambda x: f'skw_{x}_{i}'),
                data_split[i].var(axis=0).rename(index=lambda x: f'var_{x}_{i}'),
                data_split[i].min(axis=0).rename(index=lambda x: f'min_{x}_{i}')
            ])
            appended_features.append(features)

        # Combine features into final vector
        if segments is False:
            appended_features_all = pd.concat([features_whole])
        else:
            appended_features_all = pd.concat([features_whole])
            for i in range(split_index):
                appended_features_all = pd.concat([appended_features_all, appended_features[i]])

        # Add class and subject info
        appended_features_all['Y'] = wrist_class[cls]
        appended_features_all['Subject'] = subject_id

        big_list.append(appended_features_all)

    # Return list of feature vectors (one per sequence)
    return big_list


Activity labels: ['Other', 'Drink_glass', 'Drink_bottle']


## Feature Matrix Extraction
Uses gather() function to extract features from all classes. It then isolates target labels (Y) and creates the features matrix (X).

In [4]:
# ------------------------------------------------------------
# Display Configuration for DataFrames
# ------------------------------------------------------------
pd.set_option('display.max_columns', None)     # Show all columns
pd.set_option('display.max_rows', None)        # Show all rows
pd.set_option('display.max_colwidth', None)    # Don't truncate column content


# Determine if segmentation is enabled
segments = number_of_segments > 1

# ------------------------------------------------------------
# Feature Extraction from All Classes
# ------------------------------------------------------------
wrist_df = gather(wrist_class, segments=segments, split_index=number_of_segments)

# Extract target labels (Y)
wrist_Y = np.asarray(wrist_df.Y)
n_samples = len(wrist_Y)

# Determine number of features (excluding label)
num_features = len(wrist_df.columns) - 1
print('Total features:', num_features)

# ------------------------------------------------------------
# Extract Features Matrix (X)
# ------------------------------------------------------------
# NOTE: last column is assumed to be 'Subject', so it's excluded
wrist_X = np.asarray(wrist_df.iloc[:, :num_features - 1])

# Optional normalization 
"""
min_max_scaler = preprocessing.MinMaxScaler()
wrist_X = min_max_scaler.fit_transform(wrist_df.iloc[:, :num_features])
"""

# Show feature matrix dimensions and sample
print("Feature matrix shape:", wrist_X.shape)


# Show feature names used
all_features = num_features
print("Feature names:", wrist_df.iloc[:, :num_features].columns.values.tolist())


Processed class: Other
Processed class: Drink_glass
Processed class: Drink_bottle
Total features: 487
Feature matrix shape: (1000, 486)
Feature names: ['mean_x', 'mean_y', 'mean_z', 'mean_gyrox', 'mean_gyroy', 'mean_gyroz', 'mean_pitch', 'mean_roll', 'mean_yaw', 'std_x', 'std_y', 'std_z', 'std_gyrox', 'std_gyroy', 'std_gyroz', 'std_pitch', 'std_roll', 'std_yaw', 'median_x', 'median_y', 'median_z', 'median_gyrox', 'median_gyroy', 'median_gyroz', 'median_pitch', 'median_roll', 'median_yaw', 'mad_x', 'mad_y', 'mad_z', 'mad_gyrox', 'mad_gyroy', 'mad_gyroz', 'mad_pitch', 'mad_roll', 'mad_yaw', 'max_x', 'max_y', 'max_z', 'max_gyrox', 'max_gyroy', 'max_gyroz', 'max_pitch', 'max_roll', 'max_yaw', 'kur_x', 'kur_y', 'kur_z', 'kur_gyrox', 'kur_gyroy', 'kur_gyroz', 'kur_pitch', 'kur_roll', 'kur_yaw', 'skw_x', 'skw_y', 'skw_z', 'skw_gyrox', 'skw_gyroy', 'skw_gyroz', 'skw_pitch', 'skw_roll', 'skw_yaw', 'var_x', 'var_y', 'var_z', 'var_gyrox', 'var_gyroy', 'var_gyroz', 'var_pitch', 'var_roll', 'var_ya

# Classifier Setup and Performance Tracking Structures
Initializes the classifiers and sets up the data structures needed to track performance metrics during model evaluation

In [5]:
# ------------------------------------------------------------
# Classifier Setup and Performance Tracking Structures
# ------------------------------------------------------------

n = 0  # Fixed random seed

# List of classifiers (order matches `alg_array_names`)
alg_array = [
    LogisticRegression(random_state=n),
    RandomForestClassifier(n_estimators=100, verbose=0, random_state=n),
    KNeighborsClassifier(n_neighbors=3),
    GaussianNB(),
    svm.SVC(kernel='linear', C=64.0, random_state=n, probability=True),
    MLPClassifier(hidden_layer_sizes=(16, 16, 16), max_iter=1000, random_state=n),
    tree.DecisionTreeClassifier(random_state=n)
]

# Short names used for tracking each classifier
alg_array_names = ['LG', 'RF', 'KNN', 'NB', 'SVM', 'MLP', 'DT']

# ------------------------------------------------------------
# Initialize performance metric storage
# ------------------------------------------------------------

# F1 score arrays (pre-threshold and final)
f1_score_array_LG_pre, f1_score_array_RF_pre, f1_score_array_KNN_pre, f1_score_array_NB_pre, f1_score_array_SVM_pre, f1_score_array_MLP_pre, f1_score_array_DT_pre = [], [], [], [], [], [], []
f1_score_array_LG, f1_score_array_RF, f1_score_array_KNN, f1_score_array_NB, f1_score_array_SVM, f1_score_array_MLP, f1_score_array_DT = [], [], [], [], [], [], []

# Errors (raw and thresholded)
n_error_LG, n_error_RF, n_error_KNN, n_error_NB, n_error_SVM, n_error_MLP, n_error_DT = [], [], [], [], [], [], []
n_error_LG_tres, n_error_RF_tres, n_error_KNN_tres, n_error_NB_tres, n_error_SVM_tres, n_error_MLP_tres, n_error_DT_tres = [], [], [], [], [], [], []

# Thresholds
LG_tres, RF_tres, KNN_tres, NB_tres, SVM_tres, MLP_tres, DT_tres = [], [], [], [], [], [], []

# ------------------------------------------------------------
# Map metrics to variable names
# ------------------------------------------------------------

array_dict_pre = {
    "LG": f1_score_array_LG_pre,
    "RF": f1_score_array_RF_pre,
    "KNN": f1_score_array_KNN_pre,
    "NB": f1_score_array_NB_pre,
    "SVM": f1_score_array_SVM_pre,
    "MLP": f1_score_array_MLP_pre,
    "DT": f1_score_array_DT_pre
}

array_dict_pre_thres = dict(array_dict_pre)  # duplicate mapping for pre-threshold

array_dict = {
    "LG": f1_score_array_LG,
    "RF": f1_score_array_RF,
    "KNN": f1_score_array_KNN,
    "NB": f1_score_array_NB,
    "SVM": f1_score_array_SVM,
    "MLP": f1_score_array_MLP,
    "DT": f1_score_array_DT
}

error_dict = {
    "LG": n_error_LG,
    "RF": n_error_RF,
    "KNN": n_error_KNN,
    "NB": n_error_NB,
    "SVM": n_error_SVM,
    "MLP": n_error_MLP,
    "DT": n_error_DT
}

error_dict_thres = {
    "LG": n_error_LG_tres,
    "RF": n_error_RF_tres,
    "KNN": n_error_KNN_tres,
    "NB": n_error_NB_tres,
    "SVM": n_error_SVM_tres,
    "MLP": n_error_MLP_tres,
    "DT": n_error_DT_tres
}

threshold = {
    "LG": LG_tres,
    "RF": RF_tres,
    "KNN": KNN_tres,
    "NB": NB_tres,
    "SVM": SVM_tres,
    "MLP": MLP_tres,
    "DT": DT_tres
}

# ------------------------------------------------------------
# Global evaluation metric arrays
# ------------------------------------------------------------

recall_scores, precision_scores, accuracy_scores, f1_scores = [], [], [], []

# Scoring functions to use in evaluation
scoring = ['recall_macro', 'precision_macro', 'f1_macro', 'accuracy']

# Link scoring metric names to their result containers
array_score = {
    'recall_macro': recall_scores,
    'precision_macro': precision_scores,
    'f1_macro': f1_scores,
    'accuracy': accuracy_scores
}


In [6]:
# ------------------------------------------------------------
# Performance Metric Arrays for Non-Segmented Setup (NO_S)
# ------------------------------------------------------------

# F1 scores before thresholding
f1_score_array_LG_pre_NO_S, f1_score_array_RF_pre_NO_S, f1_score_array_KNN_pre_NO_S, \
f1_score_array_NB_pre_NO_S, f1_score_array_SVM_pre_NO_S, f1_score_array_MLP_pre_NO_S, \
f1_score_array_DT_pre_NO_S = [], [], [], [], [], [], []

# Classification errors (raw)
n_error_LG_NO_S, n_error_RF_NO_S, n_error_KNN_NO_S, \
n_error_NB_NO_S, n_error_SVM_NO_S, n_error_MLP_NO_S, n_error_DT_NO_S = [], [], [], [], [], [], []

# Classification errors (with thresholding)
n_error_LG_tres_NO_S, n_error_RF_tres_NO_S, n_error_KNN_tres_NO_S, \
n_error_NB_tres_NO_S, n_error_SVM_tres_NO_S, n_error_MLP_tres_NO_S, n_error_DT_tres_NO_S = [], [], [], [], [], [], []

# ------------------------------------------------------------
# Map Metrics to Classifier Names
# ------------------------------------------------------------

# Errors without thresholding
error_dict_NO_S = {
    "LG": n_error_LG_NO_S,
    "RF": n_error_RF_NO_S,
    "KNN": n_error_KNN_NO_S,
    "NB": n_error_NB_NO_S,
    "SVM": n_error_SVM_NO_S,
    "MLP": n_error_MLP_NO_S,
    "DT": n_error_DT_NO_S
}

# Errors with thresholding
error_dict_thres_NO_S = {
    "LG": n_error_LG_tres_NO_S,
    "RF": n_error_RF_tres_NO_S,
    "KNN": n_error_KNN_tres_NO_S,
    "NB": n_error_NB_tres_NO_S,
    "SVM": n_error_SVM_tres_NO_S,
    "MLP": n_error_MLP_tres_NO_S,
    "DT": n_error_DT_tres_NO_S
}

# F1 pre-threshold scores
array_dict_pre_thres_NO_S = {
    "LG": f1_score_array_LG_pre_NO_S,
    "RF": f1_score_array_RF_pre_NO_S,
    "KNN": f1_score_array_KNN_pre_NO_S,
    "NB": f1_score_array_NB_pre_NO_S,
    "SVM": f1_score_array_SVM_pre_NO_S,
    "MLP": f1_score_array_MLP_pre_NO_S,
    "DT": f1_score_array_DT_pre_NO_S
}


## Classification Setup and Utility Functions
Initializes settings for classification and defines a suite of utility functions to support model evaluation

In [7]:
# ------------------------------------------------------------
# Initial Setup
# ------------------------------------------------------------

number_classes = len(wrist_labels)
values_classes = np.unique(wrist_Y)
start = values_classes[0]

# ------------------------------------------------------------
# Utility to detect missing class indices
# ------------------------------------------------------------
def missing_elements(L, labels, Y):
    global start
    end = Y[0]
    if start == 0:
        sort = sorted(set(range(0, len(labels))).difference(L))
        for i in sort:
            if i == end:
                i = 0
    else:
        sort = sorted(set(range(1, len(labels) + 1)).difference(L))
    return sort

# ------------------------------------------------------------
# Custom LabelBinarizer supporting binary and multiclass
# ------------------------------------------------------------
class LabelBinarizer2:
    def __init__(self):
        self.lb = LabelBinarizer()

    def fit(self, X):
        X = np.array(X)
        self.lb.fit(X)
        self.classes_ = self.lb.classes_

    def fit_transform(self, X):
        X = np.array(X)
        Xlb = self.lb.fit_transform(X)
        self.classes_ = self.lb.classes_
        if len(self.classes_) == 2:
            Xlb = np.hstack((Xlb, 1 - Xlb))
        return Xlb

    def transform(self, X):
        X = np.array(X)
        Xlb = self.lb.transform(X)
        if len(self.classes_) == 2:
            Xlb = np.hstack((Xlb, 1 - Xlb))
        return Xlb

    def inverse_transform(self, Xlb):
        Xlb = np.array(Xlb)
        if len(self.classes_) == 2:
            return self.lb.inverse_transform(Xlb[:, 0])
        return self.lb.inverse_transform(Xlb)

# ------------------------------------------------------------
# Adjust predicted arrays to ensure all class columns exist
# ------------------------------------------------------------
def complete_arrays(Y, pred, pred_binarized, proba, labels):
    global start

    # Ensure predicted binarized output has all classes
    values_classes = np.unique(pred)
    miss = missing_elements(values_classes, wrist_labels, Y)
    for i in miss:
        if start != 0:
            i -= 1
        pred_binarized = np.hstack((
            pred_binarized[:, :i],
            np.zeros((pred_binarized.shape[0], 1)),
            pred_binarized[:, i:]
        ))

    # Build DataFrame of class probabilities
    values_classes = np.unique(Y)
    df = pd.DataFrame()
    columns = labels
    j = 0
    for i in values_classes:
        if start != 0:
            i -= 1
        df.insert(j, column=columns[int(i)], value=proba[:, int(i)])
        j += 1

    # Add missing columns (classes with 0 probability)
    miss = missing_elements(values_classes, columns, wrist_Y)
    for i in miss:
        if start != 0:
            i -= 1
        df.insert(int(i), column=columns[int(i)], value=np.zeros(len(proba)))

    return pred_binarized, df.values, df

# ------------------------------------------------------------
# Main classification function (log-loss based)
# ------------------------------------------------------------
def classif_model(item, n_features, X_train, X_test, y_train, y_test, log_loss_T, final_model):
    global n_samples, start
    score_selection = []

    # Feature selection
    selection = SelectKBest(f_classif, k=n_features).fit(X_train, y_train)
    X_train = selection.transform(X_train)
    X_test = selection.transform(X_test)

    # Train classifier and predict
    model = item.fit(X_train, y_train)
    pred = model.predict(X_test)
    proba = model.predict_proba(X_test)

    # Label binarization
    lb = LabelBinarizer2()
    wrist_Y_binarized = lb.fit_transform(y_test)
    pred_binarized = lb.fit_transform(pred)
    pred_binarized, proba, df = complete_arrays(y_test, pred, pred_binarized, proba, wrist_labels)

    # Compute log loss
    ll_pred = multiclass_log_loss(pred_binarized, proba)  # You must have this function defined

    # Build results DataFrame
    df3 = pd.DataFrame(pred, columns=['Pred_label'])
    df2 = pd.DataFrame(y_test, columns=['True_label'])
    df = pd.DataFrame(proba, columns=labels).join(df2).join(df3)
    df["LOG_LOSS"] = ll_pred
    df["Error_thres"] = np.where(df["Pred_label"] == df["True_label"], '_', 'TRUE')

    if not final_model:
        df_thres_NO_S = df[df.LOG_LOSS <= log_loss_T]
        df_pending_NO_S = df[df.LOG_LOSS > log_loss_T]
        n_samples = df_pending_NO_S.shape[0]
        index_selection = df_pending_NO_S.index.tolist()

        errors_selection = df_thres_NO_S[df_thres_NO_S.Error_thres == 'TRUE'].shape[0]
        if len(df_thres_NO_S):
            clasif_report = classification_report(df_thres_NO_S["True_label"], df_thres_NO_S["Pred_label"], digits=4)
            score_selection = f1_score(df_thres_NO_S["True_label"], df_thres_NO_S["Pred_label"], average="macro")
        else:
            clasif_report = []
            score_selection = 0
    else:
        df_thres_NO_S = df.copy()
        index_selection = []
        errors_selection = df_thres_NO_S[df_thres_NO_S.Error_thres == 'TRUE'].shape[0]
        score_selection = f1_score(df["True_label"], df["Pred_label"], average="macro")
        clasif_report = classification_report(df["True_label"], df["Pred_label"], digits=4)

    return index_selection, df_thres_NO_S, clasif_report, errors_selection, score_selection

# ------------------------------------------------------------
# Variant that also returns indices of classified samples
# ------------------------------------------------------------
def classif_model_active(item, n_features, X_train, X_test, y_train, y_test, log_loss_T, final_model):
    global n_samples, start
    score_selection = []

    # Feature selection
    selection = SelectKBest(chi2, k=n_features).fit(X_train, y_train)
    X_train = selection.transform(X_train)
    X_test = selection.transform(X_test)

    # Train and predict
    model = item.fit(X_train, y_train)
    pred = model.predict(X_test)
    proba = model.predict_proba(X_test)

    # Label binarization
    lb = LabelBinarizer2()
    wrist_Y_binarized = lb.fit_transform(y_test)
    pred_binarized = lb.fit_transform(pred)
    pred_binarized, proba, df = complete_arrays(y_test, pred, pred_binarized, proba, wrist_labels)

    # Compute log loss
    ll_pred = multiclass_log_loss(pred_binarized, proba)

    # Build results DataFrame
    df3 = pd.DataFrame(pred, columns=['Pred_label'])
    df2 = pd.DataFrame(y_test, columns=['True_label'])
    df = pd.DataFrame(proba, columns=labels).join(df2).join(df3)
    df["LOG_LOSS"] = ll_pred
    df["Error_thres"] = np.where(df["Pred_label"] == df["True_label"], '_', 'TRUE')

    if not final_model:
        df_thres_NO_S = df[df.LOG_LOSS <= log_loss_T]
        df_pending_NO_S = df[df.LOG_LOSS > log_loss_T]
        n_samples = df_pending_NO_S.shape[0]

        index_selection = df_pending_NO_S.index.tolist()
        index_classified = df_thres_NO_S.index.tolist()

        errors_selection = df_thres_NO_S[df_thres_NO_S.Error_thres == 'TRUE'].shape[0]
        if len(df_thres_NO_S):
            clasif_report = classification_report(df_thres_NO_S["True_label"], df_thres_NO_S["Pred_label"], digits=4)
            score_selection = f1_score(df_thres_NO_S["True_label"], df_thres_NO_S["Pred_label"], average="macro")
        else:
            clasif_report = []
            score_selection = 0
    else:
        df_thres_NO_S = df.copy()
        index_selection = []
        index_classified = df_thres_NO_S.index.tolist()
        errors_selection = df_thres_NO_S[df_thres_NO_S.Error_thres == 'TRUE'].shape[0]
        score_selection = f1_score(df["True_label"], df["Pred_label"], average="macro")
        clasif_report = classification_report(df["True_label"], df["Pred_label"], digits=4)

    return index_selection, index_classified, df_thres_NO_S, clasif_report, errors_selection, score_selection


#  Interactive Model Cascade

## Experiment and Model Configuration

Sets key parameters for the experiment:

- Proportion_test = 0.4: Specifies that 40% of the data will be used for testing. This split ratio is used when partitioning the data for evaluation (e.g., during stratified splits).

- n_times = 100: Sets the number of repetitions for the experiment.

- n_features_mX: Configurates the cascade. 


    - Model 1: 10 features
    - Model 2: 50 features
    - Model 3: All features (486)




In [8]:
# Fraction of data held out for validation out of the user instances (rest used for testing)
proportion_test = 0.4  

# Set the list of labels to be used (taken from wrist_labels)
labels = wrist_labels
print(labels)  # Print the labels for verification

# Set the number of times to repeat the experiment and the number of cross-validation folds
n_times = 100    # Number of repetitions for the experiment (for averaging results)

# ---------------------------
# Model 1 Configuration
# ---------------------------
n_features_m1 = 10   # Number of selected features for Model 1
segments_m1 = True   # Flag indicating whether Model 1 will use segmented instances

# ---------------------------
# Model 2 Configuration
# ---------------------------
n_features_m2 = 50   # Number of selected features for Model 2
segments_m2 = True   # Flag indicating whether Model 2 will use segmented instances

# ---------------------------
# Model 3 (Final Model) Configuration
# ---------------------------
n_features_m3 = 486  # Number of selected features for Model 3 (final model)
segments_m3 = True   # Flag indicating whether Model 3 will use segmented instances




['Other', 'Drink_glass', 'Drink_bottle']


## Threshold Configuration for Interactive Cascade Personalization

This cell offers three alternative configurations for setting the confidence thresholds that govern when the cascade escalates to user input. According to the paper, the system operates with base thresholds—T1 = 0.15 (for the transition from Model 1 to Model 2) and T2 = 0.40 (for Model 2 to the interactive final stage). These thresholds determine whether a prediction is accepted automatically or sent for user labeling.

User Involvement Levels:

- High Involvement (Restrictive Configuration): Uses stricter (lower) thresholds by dividing the base values (T1/1.25 and T2/1.5). With lower thresholds, the cascade is less tolerant of uncertainty, causing more instances to be escalated for user labeling. This configuration leads to increased user interaction, providing more personalized feedback that can be used to retrain the models.

- Medium Involvement (Base Configuration): Uses the base threshold values (T1 and T2). This represents a balanced scenario where the system moderately queries the user.

- Low Involvement (Permissive Configuration): Uses higher thresholds by multiplying the base values (T1 × 1.25 and T2 × 1.5). With these higher thresholds, the system becomes more tolerant of uncertainty, resulting in fewer queries to the user and consequently less direct interaction.

Usage Note:
Only one threshold configuration cell should be active at a time. This allows you to simulate a specific degree of user involvement, matching the interactive cascade strategy described in the paper. The selected configuration directly affects the frequency of user queries and, ultimately, the level of personalization achieved through retraining with user-annotated data.

In [9]:
# ------------------------------------------------------------------
# Intermediate Threshold
# ------------------------------------------------------------------
# Uncomment this cell (and comment out the others) when you want
# to use an intermediate threshold configuration.

# log_loss_T = 0.15
# log_loss_T_sel = 0.4

# file_name = "Acc_ActiveLearning_Intermediate_T1_" + str(log_loss_T) + "_T2_" + str(log_loss_T_sel) + ".xlsx"


In [10]:
# ------------------------------------------------------------------
# Permissive thress Threshold
# ------------------------------------------------------------------
# Uncomment this cell (and comment out the others) when you want
# to use a permissive threshold configuration.

# log_loss_T = 0.15 * 1.25
# log_loss_T_sel = 0.4 * 1.5

# file_name = "Acc_ActiveLearning_Permisive_T1_" + str(log_loss_T) + "_T2_" + str(log_loss_T_sel) + ".xlsx"



In [11]:
# ------------------------------------------------------------------
#restrictive Threshold
# ------------------------------------------------------------------
# Uncomment this cell (and comment out the others) when you want
# to use a restrictive threshold configuration.

log_loss_T = 0.15 / 1.25
log_loss_T_sel = 0.4 / 1.5

file_name = "Acc_ActiveLearning_Restrictive_T1_" + str(log_loss_T) + "_T2_" + str(log_loss_T_sel) + ".xlsx"


# Interactive Cascade Evaluation and Retraining

This cell implements the interactive cascade classification strategy as proposed in the paper *"Integrating Personalization into Interactive Model Cascades for Efficient Human Activity Recognition at the Edge."* The code executes a series of nested loops that simulate the cascade's processing of uncertain predictions and the subsequent integration of user-labeled data for model personalization. Below is a summary of the main steps:

1. **Cross-Validation Setup (LOSO):**  
   - Uses Leave-One-Group-Out (LOO) cross-validation, where each subject (from `wrist_df["Subject"].values`) is held out once.  
   - This simulates a user-dependent evaluation (i.e., Leave-One-Subject-Out, LOSO) as described in the paper.

2. **Baseline Performance Measurement:**  
   - For each classifier (from `alg_array` with corresponding names in `alg_array_names`), the model is trained on the training data and its baseline predictions are computed on the test set.  
   - Baseline accuracies and F1 scores are recorded for later comparison.

3. **Interactive Cascade – Round One:**  
   - The test set is further split via `StratifiedShuffleSplit` into two parts:  
     - **First Split (`X_test_first`/`y_test_first`):**  
       - Model 1 processes this subset, producing high-confidence predictions and identifying uncertain ("pending") instances.
     - **Pending Processing (Model 2):**  
       - The pending instances from Model 1 are further classified using Model 2.
     - **Final Cascade Stage (Model 3):**  
       - If uncertainty remains, Model 3 is applied, representing the final, automatic decision-making stage if no user interaction occurs.
   - The performance (accuracy of Model 1+2 and Model 3 separately) is computed and stored.

4. **User Feedback Simulation and Retraining (Round Two):**  
   - The pending instances (those that Model 1+2 could not classify confidently) are conceptually “labeled by the user” and then added to the training set.
   - The system performs a second round of training and evaluation on the updated dataset—this simulates the personalized retraining step described in the paper.
   - Again, the metrics (accuracy, number of pending instances, etc.) are recorded.

5. **Metrics Aggregation and Resetting:**  
   - Across multiple iterations (`n_times` repetitions) and subjects (via LOSO cross-validation), performance metrics such as accuracy scores and pending instance counts are averaged.
   - These results are stored in data structures (e.g., `df_results_save_final`) for later analysis, mirroring the comprehensive evaluation setup in the paper.
   - Temporary arrays for each subject’s metrics are reset after processing, ensuring each iteration starts fresh.

This evaluation procedure helps simulate the dual objectives of the paper:
- **Minimizing unnecessary user interaction:** by confidently classifying data at early stages of the cascade.
- **Enhancing personalization:** through retraining with user-supplied annotations when predictions fall below confidence thresholds.



In [ ]:
from sklearn.model_selection import LeaveOneGroupOut
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import StratifiedShuffleSplit

def logloss(true_label, predicted, eps=1e-15):
    p = np.clip(predicted, eps, 1 - eps)
    if true_label == 1:
        return -np.log(p)
    else:
        return -np.log(1 - p)

def multiclass_log_loss(y_true, y_pred, eps=1e-15):
    y_pred = np.clip(y_pred, eps, 1 - eps)
    return -(y_true * np.log(y_pred)).sum(axis=1)

best_features_num = [486]  # [1, 3, 5, 10, 20, 50, all_features] or [100, 162, 324, 486]

item_name = "LG"

subject_name_df = []
item_array_df = []
result_baseline_df = []
add_train = []  # Añadidas al train

acc_M1_M2 = []  # Acc original train original
acc_M3 = []     # Acc original train original

second_round_acc_M1_M2 = []  # Acc train new
second_round_acc_M3 = []       # Acc train new

pending_M1_M2 = []  # Pending with original train
pending_M1_M2_second_round = []  # Pending new train

appended_data = []

add_train_df = []  # Añadidas al train

acc_M1_M2_df = []  # Acc original train original
acc_M3_df = []     # Acc original train original

second_round_acc_M1_M2_df = []  # Acc train new
second_round_acc_M3_df = []      # Acc train new

pending_M1_M2_df = []  # Pending with original train
pending_M1_M2_second_round_df = []  # Pending new train

error_dict_NO_S[item_name] = []
error_dict_thres_NO_S[item_name] = []
initial_scores_M1 = []
initial_scores_M2 = []
initial_scores_M3 = []
threshold[item_name] = []
error_dict[item_name] = []
error_dict_thres[item_name] = []
array_dict_pre[item_name] = []
array_dict_pre_thres[item_name] = []
error_dict[item_name] = []
error_dict_thres[item_name] = []
array_dict_pre[item_name] = []
array_dict_pre_thres[item_name] = []

final_selection_before_M3 = []
originalclass = []
predictedclass = []
original_class = []
predicted_class = []
score_final = []
score_final_M2 = []
final_selection = []
final_selection_before_M3 = []

initial_scores_M3 = []
initial_scores_M1 = []
initial_scores_M2 = []
initial_scores_M2_all = []
index_selection_M1 = []
index_selection_M2 = []
index_selection_M3 = []

df_results_save_final = pd.DataFrame()
df_results_save = pd.DataFrame()

print(file_name)

for n_features in best_features_num:
    print('----------------------------- ')
    print('---Number of Features %0.0f-----' % n_features)
    print('----------------------------- ')
    print('\n')

    groups = wrist_df["Subject"].values
    logo = LeaveOneGroupOut()
    logo.get_n_splits(wrist_X, wrist_Y, groups)
    LeaveOneGroupOut()
    f1_array = []

    for item, item_name in zip(alg_array, alg_array_names): 
        print('\n########### ' + str(item_name) + ' ##########')
        subject_number = 1

        for train_index, test_index in logo.split(wrist_X, wrist_Y, groups):
            X_train_original, X_test_original = wrist_X[train_index], wrist_X[test_index]
            y_train_original, y_test_original = wrist_Y[train_index], wrist_Y[test_index]

            min_max_scaler = preprocessing.MinMaxScaler()
            X_train_original = min_max_scaler.fit_transform(X_train_original)
            X_test_original = min_max_scaler.transform(X_test_original)

            # Baseline
            baseline_model = item.fit(X_train_original, y_train_original)
            baseline_pred = baseline_model.predict(X_test_original)

            # DataFrame appendings
            subject_name_df.append("User " + str(subject_number))
            item_array_df.append(item_name)
            result_baseline_df.append(accuracy_score(y_test_original, baseline_pred) * 100)

            print("\n\n-------------------------- Subject " + str(subject_number) +
                  " - F1 baseline max features %.4f " % (f1_score(y_test_original, baseline_pred, average='macro')))
            print("-------------------------- Subject " + str(subject_number) +
                  " - Acc baseline max features %.4f " % (accuracy_score(y_test_original, baseline_pred)))

            min_max_scaler = preprocessing.MinMaxScaler()
            X_train_original = min_max_scaler.fit_transform(X_train_original)
            X_test_original = min_max_scaler.transform(X_test_original)

            for n in range(n_times):
                sss = StratifiedShuffleSplit(n_splits=1, test_size=proportion_test, random_state=n)
                sss.get_n_splits(X_test_original, y_test_original)

                for first_index, second_index in sss.split(X_test_original, y_test_original):
                    X_test_first, X_test_second = X_test_original[first_index], X_test_original[second_index]
                    y_test_first, y_test_second = y_test_original[first_index], y_test_original[second_index]

                    ############################################### ------------------------------ ROUND ONE
                    ###########################################################################################
                    #--------------------------- MODEL 1 ------------------------------------------------------
                    ###########################################################################################
                    index_selection, df_thres_1, clasif_report_M1, errors_selection_M1, score_selection_M1 = classif_model(
                        item, n_features_m1, X_train_original, X_test_first, y_train_original, y_test_first, log_loss_T,
                        final_model=False)
                    index_classified = df_thres_1.index.values.tolist()
                    X_test_classifed_M1 = X_test_first[index_classified]  # Classified
                    y_test_classifed_M1 = y_test_first[index_classified]  # Classified
                    X_test_selection_M1 = X_test_first[index_selection]  # pending 
                    y_test_selection_M1 = y_test_first[index_selection]  # pending 

                    ###########################################################################################
                    #--------------------------- MODEL 2 ------------------------------------------------------
                    ###########################################################################################
                    n_samples = X_test_selection_M1.shape[0]
                    if n_samples > 0:
                        index_selection, df_thres_2, clasif_report_M2, errors_selection_M2, score_selection_M2 = classif_model(
                            item, n_features_m2, X_train_original, X_test_selection_M1, y_train_original, y_test_selection_M1,
                            log_loss_T_sel, final_model=False)
                        X_test_selection_M2 = X_test_selection_M1[index_selection]  # pending 
                        y_test_selection_M2 = y_test_selection_M1[index_selection]  # pending 
                        index_classified = df_thres_2.index.values.tolist()
                        X_test_classifed_M2 = X_test_selection_M1[index_classified]  # Classified
                        y_test_classifed_M2 = y_test_selection_M1[index_classified]  # Classified
                    else:
                        print("R1 2 ----------------------------------Number of pending instances not enough to perform Model classification")
                        df_thres_2 = pd.DataFrame()
                        index_selection_M2 = index_selection
                        X_test_selection_M2 = X_test_selection_M1[index_selection]  # pending 
                        y_test_selection_M2 = y_test_selection_M1[index_selection]  # pending 

                    # Recapitulate model 2
                    M2_all = [df_thres_1, df_thres_2]
                    final_df_M2 = pd.concat(M2_all)
                    selection_M2_all = final_df_M2.shape[0]

                    ###########################################################################################
                    #--------------------------- MODEL 3 ------------------------------------------------------ 
                    ###########################################################################################
                    n_samples = X_test_selection_M2.shape[0]
                    if n_samples > 0:
                        index_selection, df_thres_3, clasif_report_M3, errors_selection_M3, score_selection_M2 = classif_model(
                            item, n_features_m3, X_train_original, X_test_selection_M2, y_train_original, y_test_selection_M2,
                            log_loss_T_sel, final_model=True)
                    else:
                        print(" R1 3 Number of pending instances not enough to perform Model3 classification")
                        df_thres_3 = pd.DataFrame()

                    ##########################################################################################
                    #--------------------------- FINAL MODEL---------------------------------------------------
                    ###########################################################################################
                    final = [df_thres_1, df_thres_2, df_thres_3]
                    final_df = pd.concat(final)
                    selection_M3 = df_thres_3.shape[0]

                    ###########################################################################################
                    #____________________________________________________________________________________________
                    #                               Second ROUND 
                    #____________________________________________________________________________________________
                    ###########################################################################################
                    # Instancias añadidas a train
                    add_train.append(X_test_selection_M2.shape[0])
                    X_train_second = np.concatenate((X_train_original, X_test_selection_M2), axis=0)
                    y_train_second = np.concatenate((y_train_original, y_test_selection_M2), axis=0)

                    ###########################################################################################
                    #--------------------------- MODEL 1 (Original Train) ------------------------------------------------------
                    ###########################################################################################
                    index_selection, df_thres_1, clasif_report_M1, errors_selection_M1, score_selection_M1 = classif_model(
                        item, n_features_m1, X_train_original, X_test_second, y_train_original, y_test_second, log_loss_T,
                        final_model=False)
                    X_test_selection_M1 = X_test_second[index_selection]  # pending 
                    y_test_selection_M1 = y_test_second[index_selection]  # pending 

                    ###########################################################################################
                    #--------------------------- MODEL 2 (Original Train) ------------------------------------------------------
                    ###########################################################################################
                    n_samples = X_test_selection_M1.shape[0]
                    if n_samples > 0:
                        index_selection, df_thres_2, clasif_report_M2, errors_selection_M2, score_selection_M2 = classif_model(
                            item, n_features_m2, X_train_original, X_test_selection_M1, y_train_original, y_test_selection_M1,
                            log_loss_T_sel, final_model=False)
                        X_test_selection_M2 = X_test_selection_M1[index_selection]  # pending 
                        y_test_selection_M2 = y_test_selection_M1[index_selection]  # pending 
                    else:
                        print("R 2 Orig Number of pending instances not enough to perform Model classification")
                        df_thres_2 = pd.DataFrame()
                        index_selection_M2 = index_selection
                        X_test_selection_M2 = X_test_selection_M1[index_selection]  # pending 
                        y_test_selection_M2 = y_test_selection_M1[index_selection]  # pending 

                    # Recapitulate model 2
                    M2_all = [df_thres_1, df_thres_2]
                    final_df_M2 = pd.concat(M2_all)

                    acc_M1_M2.append(accuracy_score(final_df_M2["True_label"].values, final_df_M2["Pred_label"].values) * 100)
                    pending_M1_M2.append(X_test_selection_M2.shape[0])

                    ###########################################################################################
                    #--------------------------- MODEL 3 (Original Train) ------------------------------------------------------
                    ###########################################################################################
                    n_samples = X_test_selection_M2.shape[0]
                    if n_samples > 0:
                        index_selection, df_thres_3, clasif_report_M3, errors_selection_M3, score_selection_M2 = classif_model(
                            item, n_features_m3, X_train_original, X_test_selection_M2, y_train_original, y_test_selection_M2,
                            log_loss_T_sel, final_model=True)
                    else:
                        print("R2 3 Orig Number of pending instances not enough to perform Model3 classification")
                        df_thres_3 = pd.DataFrame()

                    final = [df_thres_1, df_thres_2, df_thres_3]
                    final_df = pd.concat(final)
                    acc_M3.append(accuracy_score(final_df["True_label"].values, final_df["Pred_label"].values) * 100)

                    ###########################################################################################
                    #--------------------------- MODEL 1 (Personalzed Train) ------------------------------------------------------
                    ###########################################################################################
                    index_selection, df_thres_1, clasif_report_M1, errors_selection_M1, score_selection_M1 = classif_model(
                        item, n_features_m1, X_train_second, X_test_second, y_train_second, y_test_second, log_loss_T,
                        final_model=False)
                    X_test_selection_M1 = X_test_second[index_selection]  # pending 
                    y_test_selection_M1 = y_test_second[index_selection]  # pending 

                    ###########################################################################################
                    #--------------------------- MODEL 2 (Personalized Train) ------------------------------------------------------
                    ###########################################################################################
                    n_samples = X_test_selection_M1.shape[0]
                    if n_samples > 0:
                        index_selection, df_thres_2, clasif_report_M2, errors_selection_M2, score_selection_M2 = classif_model(
                            item, n_features_m2, X_train_second, X_test_selection_M1, y_train_second, y_test_selection_M1,
                            log_loss_T_sel, final_model=False)
                        X_test_selection_M2 = X_test_selection_M1[index_selection]  # pending 
                        y_test_selection_M2 = y_test_selection_M1[index_selection]  # pending 
                    else:
                        print("R2 2 New Number of pending instances not enough to perform Model classification")
                        df_thres_2 = pd.DataFrame()
                        index_selection_M2 = index_selection
                        X_test_selection_M2 = X_test_selection_M1[index_selection]  # pending 
                        y_test_selection_M2 = y_test_selection_M1[index_selection]  # pending 

                    M2_all = [df_thres_1, df_thres_2]
                    final_df_M2 = pd.concat(M2_all)

                    second_round_acc_M1_M2.append(accuracy_score(final_df_M2["True_label"].values, final_df_M2["Pred_label"].values) * 100)
                    pending_M1_M2_second_round.append(X_test_selection_M2.shape[0])

                    ###########################################################################################
                    #--------------------------- MODEL 3 (Personalized Train) ------------------------------------------------------
                    ###########################################################################################
                    n_samples = X_test_selection_M2.shape[0]
                    if n_samples > 0:
                        index_selection, df_thres_3, clasif_report_M3, errors_selection_M3, score_selection_M2 = classif_model(
                            item, n_features_m3, X_train_second, X_test_selection_M2, y_train_second, y_test_selection_M2,
                            log_loss_T_sel, final_model=True)
                    else:
                        print(" R2 3 New Number of pending instances not enough to perform Model3 classification")
                        df_thres_3 = pd.DataFrame()

                    final = [df_thres_1, df_thres_2, df_thres_3]
                    final_df = pd.concat(final)
                    second_round_acc_M3.append(accuracy_score(final_df["True_label"].values, final_df["Pred_label"].values) * 100)

            subject_number = subject_number + 1

            print("\n --- primera ronda ")
            print("Modelo 1 y 2")
            print(acc_M1_M2)
            print(np.mean(acc_M1_M2))
            print("Classificadas " + str(np.mean(pending_M1_M2)))
            print("\nModelo 3")
            print(acc_M3)
            print(np.mean(acc_M3))
            print("\n --- segunda ronda ")
            print("Modelo 1 y 2")
            print(second_round_acc_M1_M2)
            print(np.mean(second_round_acc_M1_M2))
            print("\nModelo 3")
            print(second_round_acc_M3)
            print(np.mean(second_round_acc_M3))
            print("Classificadas " + str(np.mean(pending_M1_M2_second_round)))

            add_train_df.append(np.mean(add_train))  # Añadidas al train
            acc_M1_M2_df.append(np.mean(acc_M1_M2))
            acc_M3_df.append(np.mean(acc_M3))
            second_round_acc_M1_M2_df.append(np.mean(second_round_acc_M1_M2))
            second_round_acc_M3_df.append(np.mean(second_round_acc_M3))
            pending_M1_M2_df.append(np.mean(pending_M1_M2))
            pending_M1_M2_second_round_df.append(np.mean(pending_M1_M2_second_round))

            acc_M1_M2 = []
            acc_M3 = []
            second_round_acc_M1_M2 = []
            second_round_acc_M3 = []
            pending_M1_M2 = []
            pending_M1_M2_second_round = []

            error_dict_NO_S[item_name] = []
            error_dict_thres_NO_S[item_name] = []
            initial_scores_M1 = []
            initial_scores_M2 = []
            initial_scores_M3 = []
            threshold[item_name] = []
            error_dict[item_name] = []
            error_dict_thres[item_name] = []
            array_dict_pre[item_name] = []
            array_dict_pre_thres[item_name] = []
            error_dict[item_name] = []
            error_dict_thres[item_name] = []
            array_dict_pre[item_name] = []
            array_dict_pre_thres[item_name] = []

            final_selection_before_M3 = []
            originalclass = []
            predictedclass = []
            original_class = []
            predicted_class = []
            score_final = []
            score_final_M2 = []
            final_selection = []
            final_selection_before_M3 = []

            initial_scores_M3 = []
            initial_scores_M1 = []
            initial_scores_M2 = []
            initial_scores_M2_all = []
            index_selection_M1 = []
            index_selection_M2 = []
            index_selection_M3 = []

        df_results_save["User name"] = subject_name_df
        df_results_save["Alg"] = item_array_df
        df_results_save["Acc baseline"] = result_baseline_df
        df_results_save["Add train"] = add_train_df
        df_results_save["Acc M1+M2 initial"] = acc_M1_M2_df
        df_results_save["Acc all initial"] = acc_M3_df
        df_results_save["Pending inicial"] = pending_M1_M2_df
        df_results_save["Acc M1+M2 second"] = second_round_acc_M1_M2_df
        df_results_save["Acc all second"] = second_round_acc_M3_df
        df_results_save["Pending second"] = pending_M1_M2_second_round_df

        print(item_array_df)
        appended_data.append(df_results_save)
        print("Append_data")
        print(appended_data)

        subject_name_df = []
        item_array_df = []
        result_baseline_df = []
        add_train_df = []
        acc_M1_M2_df = []
        acc_M3_df = []
        second_round_acc_M1_M2_df = []
        second_round_acc_M3_df = []
        pending_M1_M2_df = []
        pending_M1_M2_second_round_df = []

        df_results_save_final = pd.concat([df_results_save_final, df_results_save], axis=1)
        print("df_final")
        print(df_results_save_final)

    appended_data = []


Acc_ActiveLearning_Restrictive_T1_0.12_T2_0.26666666666666666.xlsx
----------------------------- 
---Number of Features 486-----
----------------------------- 



########### LG ##########


-------------------------- Subject 1 - F1 baseline max features 0.8590 
-------------------------- Subject 1 - Acc baseline max features 0.8700 


## Saving Final Results to an Excel File

In [ ]:

# Set the file name for the XLSX file
print(df_results_save_final.head())
with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
    df_results_save_final.to_excel(writer, sheet_name='Sheet1', startrow=1, header=True, index=False)

print("Saved to " + str(file_name))
